## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Pipeline for ICD-O

- **Model**: `icdo_resolver_pipeline`
- **Model Description**: This pipeline extracts oncological entities from clinical texts and map them to their corresponding ICD-O codes using `sbiobert_base_cased_mli` Sentence Bert Embeddings.

In [ ]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [8]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

In [9]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [10]:
model_name = "icdo-resolver-pipeline"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.xlarge"


### A. Create an endpoint

In [11]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

----------!

Once endpoint has been created, you would be able to perform real-time inference.

In [12]:
import json
import pandas as pd
import os
import boto3

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

def process_data_and_invoke_realtime_endpoint(data, content_type, accept):

    content_type_to_format = {'application/json': 'json', 'application/jsonlines': 'jsonl'}
    input_format = content_type_to_format.get(content_type)
    if content_type not in content_type_to_format.keys() or accept not in content_type_to_format.keys():
        raise ValueError("Invalid content_type or accept. It should be either 'application/json' or 'application/jsonlines'.")

    i = 1
    input_dir = f'inputs/real-time/{input_format}'
    output_dir = f'outputs/real-time/{input_format}'
    s3_input_dir = f"{model_name}/validation-input/real-time/{input_format}"
    s3_output_dir = f"{model_name}/validation-output/real-time/{input_format}"

    input_file_name = f'{input_dir}/input{i}.{input_format}'
    output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    while os.path.exists(input_file_name) or os.path.exists(output_file_name):
        i += 1
        input_file_name = f'{input_dir}/input{i}.{input_format}'
        output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
    os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

    input_data = json.dumps(data) if content_type == 'application/json' else data

    # Write input data to file
    with open(input_file_name, 'w') as f:
        f.write(input_data)

    # Upload input data to S3
    s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_input_dir}/{os.path.basename(input_file_name)}", Body=bytes(input_data.encode('UTF-8')))

    # Invoke the SageMaker endpoint
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=input_data,
    )

    # Read response data
    response_data = json.loads(response["Body"].read().decode("utf-8")) if accept == 'application/json' else response['Body'].read().decode('utf-8')

    # Save response data to file
    with open(output_file_name, 'w') as f_out:
        if accept == 'application/json':
            json.dump(response_data, f_out, indent=4)
        else:
            for item in response_data.split('\n'):
                f_out.write(item + '\n')

    # Upload response data to S3
    output_s3_key = f"{s3_output_dir}/{os.path.basename(output_file_name)}"
    if accept == 'application/json':
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=json.dumps(response_data).encode('UTF-8'))
    else:
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=response_data)

    return response_data

### Initial Setup

In [13]:
docs = [
    "A few studies have demonstrated that ANGPTL1 functions as a tumor suppressor gene in breast cancer  , hepatocellular carcinoma  , colorectal cancer and parathyroid carcinoma.", 
    "TRAF6 is a putative oncogene in a variety of cancers including  bladder cancer , and skin cancer. WWP2 appears to regulate the expression of the well characterized tumor suppressor phosphatase and tensin homolog (PTEN) in endometrial cancer and squamous cell carcinoma.",
]


sample_text = "TRIM50 has only been shown to act as a tumor suppressor in hepatocellular carcinoma and ovary cancer. HOXA10 exerts an oncogenic role in several tumors endometrial adenocarcinoma ."

### JSON

#### Example 1

  **Input format**:
  
  
```json
{
    "text": "Single text document"
}
```

In [14]:
input_json_data = {"text": sample_text}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
pd.DataFrame(data["predictions"])

,0,1,2,3,4,5
0,"{'ner_chunk': 'tumor', 'begin': 39, 'end': 43, 'ner_label': 'Tumor_Finding', 'ner_confidence': '0.9416', 'code': '8000/1', 'resolution': 'tumor', 'all_k_codes': '8000/1:::8040/1:::8001/1:::9365/3:::8000/6:::8103/0:::9364/3:::8940/0:::8561/0:::9230/1:::8000/3:::9365/3-C76.1:::8100/0:::8158/3:::800:::8711/0:::9135/1:::8935/1:::8010/3:::8815/1:::8960/3:::8312/3:::8153/3', 'all_k_resolutions': 'tumor:::tumorlet:::tumor cells:::askin tumor:::tumor, secondary:::pilar tumor:::ewing tumor:::mixed tumor:::warthin tumor:::codman tumor:::cancer:::askin tumor of thorax:::brooke tumor:::acth-producing tumor:::neoplasms:::glomus tumor:::dabska tumor:::stromal tumor:::carcinoma:::localized fibrous tumor:::wilms tumor:::grawitz tumor:::g cell tumor', 'all_k_distances': '0.0000:::6.2854:::7.2306:::8.0490:::8.2619:::9.2572:::9.3351:::9.5498:::9.8335:::10.1951:::10.2648:::10.3364:::10.7574:::10.8951:::10.9171:::10.9504:::10.9568:::11.1854:::11.2265:::11.2477:::11.2602:::11.2972:::11.3218'}","{'ner_chunk': 'hepatocellular', 'begin': 59, 'end': 72, 'ner_label': 'Histological_Type', 'ner_confidence': '0.9981', 'code': 'C22.0', 'resolution': 'liver', 'all_k_codes': 'C22.0:::C22:::C22.1:::8140/0-C22.0:::9080/1-C22.0:::8010/3-C22.0:::C24.0:::8140/3-C22.0:::8170/3:::C24.9:::8440/0-C22.0:::C21.2:::C18.3:::C71:::C42.3:::C75.9:::C42.2:::C26.0:::8576/3:::8001/3-C22.0:::8000/3-C22.0:::8800/3-C22.0:::C25.3', 'all_k_resolutions': 'liver:::liver and intrahepatic bile ducts:::intrahepatic bile duct:::adenoma, of liver:::teratoma, of liver:::carcinoma, of liver:::extrahepatic bile duct:::adenocarcinoma, of liver:::hepatoma:::billiary tract:::cystadenoma, of liver:::cloacogenic zone:::hepatic flexure of colon:::brain:::reticuloendothelial system:::endocrine gland:::spleen:::intestinal tract:::hepatoid carcinoma:::tumor cells, malignant of liver:::neoplasm, malignant of liver:::sarcoma, of liver:::pancreatic duct', 'all_k_distances': '8.9670:::9.5345:::10.0203:::10.5846:::11.4080:::11.4539:::11.5473:::11.7515:::12.0136:::12.1413:::12.1659:::12.4287:::12.5463:::12.5747:::12.5802:::12.6025:::12.6953:::12.6954:::12.8245:::12.8977:::12.9020:::12.9039:::12.9249'}","{'ner_chunk': 'carcinoma', 'begin': 74, 'end': 82, 'ner_label': 'Cancer_Dx', 'ner_confidence': '0.9491', 'code': '8010/3', 'resolution': 'carcinoma', 'all_k_codes': '8010/3:::8010/9:::8420/3:::8480/3:::8240/3:::8550/3:::8140/3:::8010/6:::8530/3:::8980/3:::8070/3:::8010/3-C06.9:::8054/3:::8520/3:::8010/2:::8244/3:::8141/3:::8051/3:::8575/3:::8481/3:::8110/3:::8441/3', 'all_k_resolutions': 'carcinoma:::carcinomatosis:::ceruminous carcinoma:::mucous carcinoma:::carcinoid:::acinar carcinoma:::adenocarcinoma:::secondary carcinoma:::inflammatory carcinoma:::carcinosarcoma:::squamous carcinoma:::carcinoma, of mouth:::warty carcinoma:::lobular carcinoma:::carcinoma in situ:::composite carcinoid:::scirrhous carcinoma:::verrucous carcinoma:::metaplastic carcinoma:::mucin-producing carcinoma:::matrical carcinoma:::serous carcinoma', 'all_k_distances': '0.0000:::3.8145:::6.1551:::6.1916:::6.2402:::6.7504:::6.9016:::6.9301:::7.1193:::7.1926:::7.2897:::7.3397:::7.4187:::7.4689:::7.4910:::7.6357:::7.6536:::7.7765:::7.7787:::7.7825:::7.8719:::7.8739'}","{'ner_chunk': 'ovary cancer', 'begin': 88, 'end': 99, 'ner_label': 'Oncological', 'ner_confidence': '0.84344995', 'code': '8010/3-C56.9', 'resolution': 'carcinoma, of ovary', 'all_k_codes': '8010/3-C56.9:::8140/3-C56.9:::8980/3-C56.9:::8441/3-C56.9:::8230/3-C56.9:::8051/3-C56.9:::8560/3-C56.9:::8021/3-C56.9:::8010/2-C56.9:::8510/3-C56.9:::8440/3-C56.9:::8050/3-C56.9:::8410/3-C56.9:::8575/3-C56.9:::8070/3-C56.9:::8262/3-C56.9:::8474/3-C56.9:::8140/2-C56.9:::8144/3-C56.9:::9081/3-C56.9:::8640/3-C56.9:::8246/3-C56.9:::9070/3-C56.9:::8245/3-C56.9:::8141/3-C56.9', 'all_k_resolutions': 'carcinoma, of ovary:::adenocarcinoma, of ovary:::carcinosarcoma, of ovary:::serous carcinoma, of ovary:::solid carcinoma, of ovary:::verrucous carcinoma, of ovary:::adenosquamous ca

#### Example 2

  **Input format**:
  
  
```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ]
}
```

In [15]:
input_json_data = {"text": docs}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
pd.DataFrame(data["predictions"])

,0,1,2,3,4,5,6
0,"{'ner_chunk': 'tumor', 'begin': 60, 'end': 64, 'ner_label': 'Tumor_Finding', 'ner_confidence': '0.9633', 'code': '8000/1', 'resolution': 'tumor', 'all_k_codes': '8000/1:::8040/1:::8001/1:::9365/3:::8000/6:::8103/0:::9364/3:::8940/0:::8561/0:::9230/1:::8000/3:::9365/3-C76.1:::8100/0:::8158/3:::800:::8711/0:::9135/1:::8935/1:::8010/3:::8815/1:::8960/3:::8312/3:::8153/3', 'all_k_resolutions': 'tumor:::tumorlet:::tumor cells:::askin tumor:::tumor, secondary:::pilar tumor:::ewing tumor:::mixed tumor:::warthin tumor:::codman tumor:::cancer:::askin tumor of thorax:::brooke tumor:::acth-producing tumor:::neoplasms:::glomus tumor:::dabska tumor:::stromal tumor:::carcinoma:::localized fibrous tumor:::wilms tumor:::grawitz tumor:::g cell tumor', 'all_k_distances': '0.0000:::6.2854:::7.2306:::8.0490:::8.2619:::9.2572:::9.3351:::9.5498:::9.8335:::10.1951:::10.2648:::10.3364:::10.7574:::10.8951:::10.9171:::10.9504:::10.9568:::11.1854:::11.2265:::11.2477:::11.2602:::11.2972:::11.3218'}","{'ner_chunk': 'breast cancer', 'begin': 85, 'end': 97, 'ner_label': 'Oncological', 'ner_confidence': '0.939', 'code': '8010/3-C50.9', 'resolution': 'carcinoma, of breast', 'all_k_codes': '8010/3-C50.9:::8140/3-C50.9:::8980/3-C50.9:::8230/3-C50.9:::8520/3-C50.9:::8575/3-C50.9:::8510/3-C50.9:::8051/3-C50.9:::8440/3-C50.9:::8201/3-C50.9:::8410/3-C50.9:::8070/3-C50.9:::8050/3-C50.9:::8550/3-C50.9:::8530/3-C50.9:::8560/3-C50.9:::8501/3-C50.9:::8800/3-C50.9:::8810/3-C50.9:::8010/2-C50.9:::8010/3-C50.6:::8145/3-C50.9:::8500/3-C50.9:::8021/3-C50.9:::9010/0-C50.9', 'all_k_resolutions': 'carcinoma, of breast:::adenocarcinoma, of breast:::carcinosarcoma, of breast:::solid carcinoma, of breast:::lobular carcinoma, of breast:::metaplastic carcinoma, of breast:::medullary carcinoma, of breast:::verrucous carcinoma, of breast:::cystadenocarcinoma, of breast:::cribriform carcinoma, of breast:::sebaceous carcinoma of breast:::squamous cell carcinoma, of breast:::papillary carcinoma, of breast:::acinar cell carcinoma of breast:::inflammatory carcinoma of breast:::adenosquamous carcinoma of breast:::comedocarcinoma, of breast:::sarcoma, of breast:::fibrosarcoma, of breast:::carcinoma in situ, of breast:::carcinoma, of axillary tail of breast:::carcinoma, diffuse type of breast:::infiltrating duct carcinoma, of breast:::carcinoma, anaplastic, of breast:::fibroadenoma, of breast', 'all_k_distances': '5.7740:::6.7579:::7.2199:::7.2368:::7.2713:::7.4459:::7.4703:::7.5246:::7.7778:::7.8426:::7.8444:::7.8521:::7.8565:::7.8621:::7.8664:::7.8862:::7.8897:::7.9308:::8.0135:::8.1355:::8.2741:::8.2747:::8.3045:::8.3297:::8.3516'}","{'ner_chunk': 'hepatocellular', 'begin': 102, 'end': 115, 'ner_label': 'Histological_Type', 'ner_confidence': '0.9995', 'code': 'C22.0', 'resolution': 'liver', 'all_k_codes': 'C22.0:::C22:::C22.1:::8140/0-C22.0:::9080/1-C22.0:::8010/3-C22.0:::C24.0:::8140/3-C22.0:::8170/3:::C24.9:::8440/0-C22.0:::C21.2:::C18.3:::C71:::C42.3:::C75.9:::C42.2:::C26.0:::8576/3:::8001/3-C22.0:::8000/3-C22.0:::8800/3-C22.0:::C25.3', 'all_k_resolutions': 'liver:::liver and intrahepatic bile ducts:::intrahepatic bile duct:::adenoma, of liver:::teratoma, of liver:::carcinoma, of liver:::extrahepatic bile duct:::adenocarcinoma, of liver:::hepatoma:::billiary tract:::cystadenoma, of liver:::cloacogenic zone:::hepatic flexure of colon:::brain:::reticuloendothelial system:::endocrine gland:::spleen:::intestinal tract:::hepatoid carcinoma:::tumor cells, malignant of liver:::neoplasm, malignant of liver:::sarcoma, of liver:::pancreatic duct', 'all_k_distances': '8.9670:::9.5345:::10.0203:::10.5846:::11.4080:::11.4539:::11.5473:::11.7515:::12.0136:::12.1413:::12.1659:::12.4287:::12.5463:::12.5747:::12.5802:::12.6025:::12.6953:::12.6954:::12.8245:::12.8977:::12.9020:::12.9039:::12.9249'}","{'ner_chunk': 'carcinoma', 'begin': 117, 'end': 125, 'ner_label': 'Cancer_Dx', 'ner_confidence': '0.971', 'code': '8010/3', 'resolution': 'carcinoma', 'all_k_codes': '8010/3:::8010/9:::8420

### JSON Lines

In [16]:
import json

def create_jsonl(records):
    json_records = []

    for text in records:
        record = {
            "text": text
        }
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record) for record in json_records)

    return json_lines

input_jsonl_data = create_jsonl(docs)

#### Example 1

  **Input format**:
  
```json
{"text": "Text document 1"}
{"text": "Text document 2"}
```

In [17]:
data = process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"predictions": [{"ner_chunk": "tumor", "begin": 60, "end": 64, "ner_label": "Tumor_Finding", "ner_confidence": "0.9633", "code": "8000/1", "resolution": "tumor", "all_k_codes": "8000/1:::8040/1:::8001/1:::9365/3:::8000/6:::8103/0:::9364/3:::8940/0:::8561/0:::9230/1:::8000/3:::9365/3-C76.1:::8100/0:::8158/3:::800:::8711/0:::9135/1:::8935/1:::8010/3:::8815/1:::8960/3:::8312/3:::8153/3", "all_k_resolutions": "tumor:::tumorlet:::tumor cells:::askin tumor:::tumor, secondary:::pilar tumor:::ewing tumor:::mixed tumor:::warthin tumor:::codman tumor:::cancer:::askin tumor of thorax:::brooke tumor:::acth-producing tumor:::neoplasms:::glomus tumor:::dabska tumor:::stromal tumor:::carcinoma:::localized fibrous tumor:::wilms tumor:::grawitz tumor:::g cell tumor", "all_k_distances": "0.0000:::6.2854:::7.2306:::8.0490:::8.2619:::9.2572:::9.3351:::9.5498:::9.8335:::10.1951:::10.2648:::10.3364:::10.7574:::10.8951:::10.9171:::10.9504:::10.9568:::11.1854:::11.2265:::11.2477:::11.2602:::11.2972:::11.3218

### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [18]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Batch inference

In [19]:
import json
import os

input_dir = 'inputs/batch'
json_input_dir = f"{input_dir}/json"
jsonl_input_dir = f"{input_dir}/jsonl"

output_dir = 'outputs/batch'
json_output_dir = f"{output_dir}/json"
jsonl_output_dir = f"{output_dir}/jsonl"

os.makedirs(json_input_dir, exist_ok=True)
os.makedirs(jsonl_input_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(jsonl_output_dir, exist_ok=True)

validation_json_file_name = "input.json"

validation_jsonl_file_name = "input.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/jsonl/"

def write_and_upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    if file_format == ".json":
        input_data = json.dumps(input_data)

    with open(file_name, "w") as f:
        f.write(input_data)

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/batch/{file_format[1:]}/{os.path.basename(file_name)}",
        Body=(bytes(input_data.encode("UTF-8"))),
    )

In [20]:
input_jsonl_data = create_jsonl(docs)
input_json_data = {"text": docs}

write_and_upload_to_s3(input_json_data, f"{json_input_dir}/{validation_json_file_name}")

write_and_upload_to_s3(input_jsonl_data, f"{jsonl_input_dir}/{validation_jsonl_file_name}")

### JSON

In [21]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

INFO:sagemaker:Creating transform job with name: icdo-resolver-pipeline-en-2024-12-03-10-27-47-864


........................................24/12/03 10:34:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).

#015[Stage 0:>                                                          (0 + 1) / 1]#015#015                                                                                #015INFO:     Started server process [7]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)
📋 Loading license number 0 from /root/.johnsnowlabs/licenses/license_number_{number}_for_Spark-Healthcare_Spark-OCR.json
👌 Launched #033cpu optimized#033 session with with: 🚀Spark-NLP==5.5.0, 💊Spark-Healthcare==5.5.0, running on ⚡ PySpark==3.4.0
INFO:     169.254.255.130:57260 - "GET /ping HTTP/1.1" 200 OK
INFO:     

In [22]:
from urllib.parse import urlparse

def process_s3_json_output_and_save(validation_file_name):

    output_file_path = f"{json_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    df = pd.DataFrame(data["predictions"])
    display(df)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        json.dump(data, f_out, indent=4)

In [23]:
process_s3_json_output_and_save(validation_json_file_name)

,0,1,2,3,4,5,6
0,"{'ner_chunk': 'tumor', 'begin': 60, 'end': 64, 'ner_label': 'Tumor_Finding', 'ner_confidence': '0.9633', 'code': '8000/1', 'resolution': 'tumor', 'all_k_codes': '8000/1:::8040/1:::8001/1:::9365/3:::8000/6:::8103/0:::9364/3:::8940/0:::8561/0:::9230/1:::8000/3:::9365/3-C76.1:::8100/0:::8158/3:::800:::8711/0:::9135/1:::8935/1:::8010/3:::8815/1:::8960/3:::8312/3:::8153/3', 'all_k_resolutions': 'tumor:::tumorlet:::tumor cells:::askin tumor:::tumor, secondary:::pilar tumor:::ewing tumor:::mixed tumor:::warthin tumor:::codman tumor:::cancer:::askin tumor of thorax:::brooke tumor:::acth-producing tumor:::neoplasms:::glomus tumor:::dabska tumor:::stromal tumor:::carcinoma:::localized fibrous tumor:::wilms tumor:::grawitz tumor:::g cell tumor', 'all_k_distances': '0.0000:::6.2854:::7.2306:::8.0490:::8.2619:::9.2572:::9.3351:::9.5498:::9.8335:::10.1951:::10.2648:::10.3364:::10.7574:::10.8951:::10.9171:::10.9504:::10.9568:::11.1854:::11.2265:::11.2477:::11.2602:::11.2972:::11.3218'}","{'ner_chunk': 'breast cancer', 'begin': 85, 'end': 97, 'ner_label': 'Oncological', 'ner_confidence': '0.939', 'code': '8010/3-C50.9', 'resolution': 'carcinoma, of breast', 'all_k_codes': '8010/3-C50.9:::8140/3-C50.9:::8980/3-C50.9:::8230/3-C50.9:::8520/3-C50.9:::8575/3-C50.9:::8510/3-C50.9:::8051/3-C50.9:::8440/3-C50.9:::8201/3-C50.9:::8410/3-C50.9:::8070/3-C50.9:::8050/3-C50.9:::8550/3-C50.9:::8530/3-C50.9:::8560/3-C50.9:::8501/3-C50.9:::8800/3-C50.9:::8810/3-C50.9:::8010/2-C50.9:::8010/3-C50.6:::8145/3-C50.9:::8500/3-C50.9:::8021/3-C50.9:::9010/0-C50.9', 'all_k_resolutions': 'carcinoma, of breast:::adenocarcinoma, of breast:::carcinosarcoma, of breast:::solid carcinoma, of breast:::lobular carcinoma, of breast:::metaplastic carcinoma, of breast:::medullary carcinoma, of breast:::verrucous carcinoma, of breast:::cystadenocarcinoma, of breast:::cribriform carcinoma, of breast:::sebaceous carcinoma of breast:::squamous cell carcinoma, of breast:::papillary carcinoma, of breast:::acinar cell carcinoma of breast:::inflammatory carcinoma of breast:::adenosquamous carcinoma of breast:::comedocarcinoma, of breast:::sarcoma, of breast:::fibrosarcoma, of breast:::carcinoma in situ, of breast:::carcinoma, of axillary tail of breast:::carcinoma, diffuse type of breast:::infiltrating duct carcinoma, of breast:::carcinoma, anaplastic, of breast:::fibroadenoma, of breast', 'all_k_distances': '5.7740:::6.7579:::7.2199:::7.2368:::7.2713:::7.4459:::7.4703:::7.5246:::7.7778:::7.8426:::7.8444:::7.8521:::7.8565:::7.8621:::7.8664:::7.8862:::7.8897:::7.9308:::8.0135:::8.1355:::8.2741:::8.2747:::8.3045:::8.3297:::8.3516'}","{'ner_chunk': 'hepatocellular', 'begin': 102, 'end': 115, 'ner_label': 'Histological_Type', 'ner_confidence': '0.9995', 'code': 'C22.0', 'resolution': 'liver', 'all_k_codes': 'C22.0:::C22:::C22.1:::8140/0-C22.0:::9080/1-C22.0:::8010/3-C22.0:::C24.0:::8140/3-C22.0:::8170/3:::C24.9:::8440/0-C22.0:::C21.2:::C18.3:::C71:::C42.3:::C75.9:::C42.2:::C26.0:::8576/3:::8001/3-C22.0:::8000/3-C22.0:::8800/3-C22.0:::C25.3', 'all_k_resolutions': 'liver:::liver and intrahepatic bile ducts:::intrahepatic bile duct:::adenoma, of liver:::teratoma, of liver:::carcinoma, of liver:::extrahepatic bile duct:::adenocarcinoma, of liver:::hepatoma:::billiary tract:::cystadenoma, of liver:::cloacogenic zone:::hepatic flexure of colon:::brain:::reticuloendothelial system:::endocrine gland:::spleen:::intestinal tract:::hepatoid carcinoma:::tumor cells, malignant of liver:::neoplasm, malignant of liver:::sarcoma, of liver:::pancreatic duct', 'all_k_distances': '8.9670:::9.5345:::10.0203:::10.5846:::11.4080:::11.4539:::11.5473:::11.7515:::12.0136:::12.1413:::12.1659:::12.4287:::12.5463:::12.5747:::12.5802:::12.6025:::12.6953:::12.6954:::12.8245:::12.8977:::12.9020:::12.9039:::12.9249'}","{'ner_chunk': 'carcinoma', 'begin': 117, 'end': 125, 'ner_label': 'Cancer_Dx', 'ner_confidence': '0.971', 'code': '8010/3', 'resolution': 'carcinoma', 'all_k_codes': '8010/3:::8010/9:::8420

### JSON Lines

In [24]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

INFO:sagemaker:Creating model with name: icdo-resolver-pipeline-en-2024-12-03-10-36-16-203
INFO:sagemaker:Creating transform job with name: icdo-resolver-pipeline-en-2024-12-03-10-36-16-939


............................................24/12/03 10:43:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
#015[Stage 0:>                                                          (0 + 1) / 1]#015#015                                                                                #015#015[Stage 5:>                                                          (0 + 0) / 1]#015#015                                                                                #015#015[Stage 23:>                                                         (0 + 0) / 4]#015#015                                                                                #015INFO:     Started server process [7]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http:/

In [25]:
from urllib.parse import urlparse

def process_s3_jsonlines_output_and_save(validation_file_name):

    output_file_path = f"{jsonl_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        for item in data.split('\n'):
            f_out.write(item + '\n')

In [26]:
process_s3_jsonlines_output_and_save(validation_jsonl_file_name)

{"predictions": [{"ner_chunk": "tumor", "begin": 60, "end": 64, "ner_label": "Tumor_Finding", "ner_confidence": "0.9633", "code": "8000/1", "resolution": "tumor", "all_k_codes": "8000/1:::8040/1:::8001/1:::9365/3:::8000/6:::8103/0:::9364/3:::8940/0:::8561/0:::9230/1:::8000/3:::9365/3-C76.1:::8100/0:::8158/3:::800:::8711/0:::9135/1:::8935/1:::8010/3:::8815/1:::8960/3:::8312/3:::8153/3", "all_k_resolutions": "tumor:::tumorlet:::tumor cells:::askin tumor:::tumor, secondary:::pilar tumor:::ewing tumor:::mixed tumor:::warthin tumor:::codman tumor:::cancer:::askin tumor of thorax:::brooke tumor:::acth-producing tumor:::neoplasms:::glomus tumor:::dabska tumor:::stromal tumor:::carcinoma:::localized fibrous tumor:::wilms tumor:::grawitz tumor:::g cell tumor", "all_k_distances": "0.0000:::6.2854:::7.2306:::8.0490:::8.2619:::9.2572:::9.3351:::9.5498:::9.8335:::10.1951:::10.2648:::10.3364:::10.7574:::10.8951:::10.9171:::10.9504:::10.9568:::11.1854:::11.2265:::11.2477:::11.2602:::11.2972:::11.3218

In [27]:
model.delete_model()

INFO:sagemaker:Deleting model with name: icdo-resolver-pipeline-en-2024-12-03-10-36-16-203


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

